# HiCo Layer 1

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import join
import cv2
import pandas as pd
import os
import random as rn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

In [2]:
SEED = 321
rn.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
NUM_TN = 10
SUB_REGION_SCALE = 14
MIN_SUB_REGION_SCALE = 12
MAX_SUB_REGION_SCALE = 16
NUM_HICO_LAYER = 5

#dataset specific parameters
NUM_CLASS = 5

In [4]:
data = "../input/flowers-recognition/flowers/"
folders = os.listdir(data)

In [5]:
image_names = []
train_labels = []
train_images = []

size = 32,32

for folder in folders:
    for file in os.listdir(os.path.join(data,folder)):
        if file.endswith("jpg"):
            image_names.append(os.path.join(data,folder,file))
            train_labels.append(folder)
            img = cv2.imread(os.path.join(data,folder,file))
            im = cv2.resize(img,size)
            train_images.append(im)
        else:
            continue

In [6]:
train = np.array(train_images)
train = train.astype('float32')
train.shape

(4323, 32, 32, 3)

In [7]:
# Train, Val, Test split = 0.8, 0.1, 0.1 of the dataset
X_train,X_val,y_train,y_val = train_test_split(train,train_labels, test_size = 0.2)
X_val,X_test,y_val,y_test = train_test_split(X_val,y_val, test_size = 0.5)

## Data Pre-Processing

In [8]:
#Generate cropped train image
X_train_cropped_list = []
coordinate_list = []
scale_list = []
for i in range(NUM_TN):
  X_train_cropped = []
  sub_region_scale = rn.randint(MIN_SUB_REGION_SCALE, MAX_SUB_REGION_SCALE)
  x = rn.randint(0, X_train.shape[1] - sub_region_scale)
  y = rn.randint(0, X_train.shape[1] - sub_region_scale)
  coordinate = (x, y)
  for j in range(X_train.shape[0]):
    image_cropped = []
    for row in range(y, y + sub_region_scale):
      image_cropped_row = []
      for column in range(x, x + sub_region_scale):
        image_cropped_column = []
        for rgb in range(3):
          image_cropped_column.append(X_train[j][row][column][rgb])
        image_cropped_row.append(image_cropped_column)
      image_cropped.append(image_cropped_row)
    X_train_cropped.append(image_cropped)
  
  X_train_cropped = np.array(X_train_cropped)
  X_train_cropped_list.append(X_train_cropped)
  coordinate_list.append(coordinate)
  scale_list.append(sub_region_scale)

In [9]:
#Generate cropped test image
X_val_cropped_list = []
for i in range(NUM_TN):
  X_val_cropped = []
  sub_region_scale = scale_list[i]
  x = coordinate_list[i][0]
  y = coordinate_list[i][1]
  for j in range(X_val.shape[0]):
    image_cropped = []
    for row in range(y, y + sub_region_scale):
      image_cropped_row = []
      for column in range(x, x + sub_region_scale):
        image_cropped_column = []
        for rgb in range(3):
          image_cropped_column.append(X_val[j][row][column][rgb])
        image_cropped_row.append(image_cropped_column)
      image_cropped.append(image_cropped_row)
    X_val_cropped.append(image_cropped)

  X_val_cropped = np.array(X_val_cropped)
  X_val_cropped_list.append(X_val_cropped)

In [10]:
#Generate cropped test image
X_test_cropped_list = []
for i in range(NUM_TN):
  X_test_cropped = []
  sub_region_scale = scale_list[i]
  x = coordinate_list[i][0]
  y = coordinate_list[i][1]
  for j in range(X_test.shape[0]):
    image_cropped = []
    for row in range(y, y + sub_region_scale):
      image_cropped_row = []
      for column in range(x, x + sub_region_scale):
        image_cropped_column = []
        for rgb in range(3):
          image_cropped_column.append(X_test[j][row][column][rgb])
        image_cropped_row.append(image_cropped_column)
      image_cropped.append(image_cropped_row)
    X_test_cropped.append(image_cropped)

  X_test_cropped = np.array(X_test_cropped)
  X_test_cropped_list.append(X_test_cropped)

In [11]:
#rescaling dataset
for i in range(NUM_TN):
  temp = []
  for j in range (X_train.shape[0]):
    temp.append(cv2.resize(X_train_cropped_list[i][j], (SUB_REGION_SCALE, SUB_REGION_SCALE)))
  temp = np.array(temp)
  X_train_cropped_list[i] = temp
    
  temp = []
  for j in range(X_val.shape[0]):
    temp.append(cv2.resize(X_val_cropped_list[i][j], (SUB_REGION_SCALE, SUB_REGION_SCALE)))
  temp = np.array(temp)
  X_val_cropped_list[i] = temp

  temp = []
  for j in range(X_test.shape[0]):
    temp.append(cv2.resize(X_test_cropped_list[i][j], (SUB_REGION_SCALE, SUB_REGION_SCALE)))
  temp = np.array(temp)
  X_test_cropped_list[i] = temp

In [12]:
#normalizing dataset
for i in range(NUM_TN):
    X_train_cropped_list[i] = X_train_cropped_list[i]/255
    X_val_cropped_list[i] = X_val_cropped_list[i]/255
    X_test_cropped_list[i] = X_test_cropped_list[i]/255

In [13]:
#transform to vector
for i in range(NUM_TN):
    X_train_cropped_list[i] = X_train_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE*3))
    X_val_cropped_list[i] = X_val_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE*3))
    X_test_cropped_list[i] = X_test_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE*3))

In [14]:
#one-hot encoding
y_train_one_hot = pd.get_dummies(y_train)
y_train_one_hot  = y_train_one_hot .values.argmax(1)
y_train_one_hot  = to_categorical(y_train_one_hot)
    
y_val_one_hot = pd.get_dummies(y_val)
y_val_one_hot  = y_val_one_hot .values.argmax(1)
y_val_one_hot  = to_categorical(y_val_one_hot)

y_test_one_hot = pd.get_dummies(y_test)
y_test_one_hot  = y_test_one_hot .values.argmax(1)
y_test_one_hot  = to_categorical(y_test_one_hot)

## HiCo Layer 1

In [15]:
#build ANN model
ensemble = []
for i in range(NUM_TN):
  model = Sequential()
  model.add(Dense(64, activation='relu', input_dim=SUB_REGION_SCALE*SUB_REGION_SCALE*X_train.shape[3]))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(NUM_CLASS, activation = 'softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  ensemble.append((model, i, coordinate_list[i], scale_list[i]))

In [16]:
#train model
history = []
for i in range(NUM_TN):
    print()
    print('Model %d' %i)
    print()
    hist = ensemble[i][0].fit(X_train_cropped_list[i], y_train_one_hot, validation_data=(X_test_cropped_list[i], y_test_one_hot), epochs=5, batch_size=128)
    history.append(hist)


Model 0

Epoch 1/5
28/28 [==============================] - 2s 23ms/step - loss: 1.5881 - accuracy: 0.2558 - val_loss: 1.4359 - val_accuracy: 0.3857
Epoch 2/5
28/28 [==============================] - 0s 4ms/step - loss: 1.4081 - accuracy: 0.3797 - val_loss: 1.3685 - val_accuracy: 0.3857
Epoch 3/5
28/28 [==============================] - 0s 4ms/step - loss: 1.3705 - accuracy: 0.3934 - val_loss: 1.3369 - val_accuracy: 0.4296
Epoch 4/5
28/28 [==============================] - 0s 4ms/step - loss: 1.3234 - accuracy: 0.4131 - val_loss: 1.4045 - val_accuracy: 0.3764
Epoch 5/5
28/28 [==============================] - 0s 4ms/step - loss: 1.3561 - accuracy: 0.3971 - val_loss: 1.3382 - val_accuracy: 0.4088

Model 1

Epoch 1/5
28/28 [==============================] - 1s 9ms/step - loss: 1.5522 - accuracy: 0.2715 - val_loss: 1.4588 - val_accuracy: 0.3557
Epoch 2/5
28/28 [==============================] - 0s 4ms/step - loss: 1.4078 - accuracy: 0.4002 - val_loss: 1.3882 - val_accuracy: 0.4180
Epoch 

In [17]:
if not os.path.isdir('./fyp') :
    os.mkdir('./fyp')

with open('./fyp/CIFAR-10.pickle', 'wb') as f:
    pickle.dump([X_train_cropped_list, y_train_one_hot, X_val_cropped_list, y_val_one_hot, X_test_cropped_list, y_test_one_hot, coordinate_list, scale_list], f)

In [18]:
for i in range(NUM_TN):
    ensemble[i][0].save_weights('./fyp/CIFAR-10_' + str(i) + '.h5')